In [16]:
import time
import random
import pandas as pd
import numpy as np
import copy
import math
import sklearn

# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import accuracy_score

### Load the data from the dataframe

In [17]:
dataframe = pd.read_csv('./processed_data.csv')

In [18]:
print(dataframe.shape)

(891, 8)


These functions are used to print the unique values of every feature, and the number of missing values for each feature. The loaded dataset is already preprocessed, so all the values are numbers between 0 and 1 and there are no missing values.

In [19]:
# GETTER FOR UNIQUE VALUES
def get_uniques(dataframe):
    for feature in dataframe.columns:

        print(f"Feature {feature}: \n{list(pd.Series(dataframe[feature]).unique())}")

In [20]:
# GETTER FOR MISSING VALUES
def get_missing_values(dataframe):
    for col in dataframe.isnull():
        all = list()
        print(f'Feature {col}:\n\n')
        for index, instance in enumerate(dataframe.isnull()[col]):
            if instance is True:
                all.append(index)
        print(f'Missing instances: {all}')
        print('Number of missing instances: ', len(all))

In [21]:
# # GET UNIQUE VALUES
# print(get_uniques(dataframe))

# # GET MISSING VALUE
# print(get_missing_values(dataframe))

### Define functions for calculating metrics

In [22]:
from keras import backend as K

def recall_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_fun(y_true, y_pred):
    precision = precision_fun(y_true, y_pred)
    recall = recall_fun(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Define functions to be used for the Genetic Algorithm

In [23]:
from sklearn.model_selection import train_test_split


def generate_dataframes_for_training(dataframe, split_frac=0.8):
    """
    Generates training and testing dataframes from a complete dataframe, according to the split_frac parameter
    """
    X = dataframe.loc[:, dataframe.columns != 'Survived']
    y = dataframe.loc[:, dataframe.columns == 'Survived']

    return train_test_split(X, y, test_size = 0.2, random_state=42, stratify= y)
    
    # return X_train, X_test, y_train, y_test

In [24]:
def get_n_individual(counter, population):
    """
    If counter is 0, return the individual with the highest prob
    If counter is 1, return the second individual with the highest prob
    If counter is 2, return the third individual withthe highest prob
    """
    index = counter + 1
    probabilities = [ind[1] for ind in population]
    sorted_probs = sorted(probabilities, key=float)
    max_prob = sorted_probs[-index]
    max_individual = [ind[0] for ind in population if ind[1] == max_prob][0]
    
    return max_individual

In [25]:

def generate_random_individuals(num_individuals, num_features, max_features=None, verbose=False):
    """
    Randomly generates individuals

    The number of individuals to generate is given by the num_individuals parameter
    The length of each individual is equal to the num_features parameter
    The maximum number of active features for every individual is given by the max_features parameter
    """
    if verbose: print('GENERATING RANDOM INDIVIDUALS.... ')
        
    individuals = list()
    
    for _ in range(num_individuals):
        individual = ''
        for col in range(num_features):
            # For each char in the individual, a 1 or a 0 is randomly generated
            if individual.count('1') == max_features:
                individual += '0'
                continue
                
            individual += str(random.randint(0, 1))
            
        if verbose: print(f'Genrated a new indivudal: {individual}')
        individuals.append(individual)
        
    if verbose: print(f'Generated list of {num_individuals} individuals: {individuals}')
        
    return individuals

In [26]:

def create_model(X, y, verbose=False):
    """
    X: training dataset to be used. Its shape is used to set the input shape for the model.
    """
    
    from sklearn.ensemble import RandomForestClassifier

    clf_forest = RandomForestClassifier(n_estimators=100, max_depth=10)
    clf_forest.fit(X, y)
    
    if verbose: print('MODEL SUMMARY: \n')
    if verbose: print(clf_forest.summary())
        
    return clf_forest



def get_weights(population):
    """
    Calculate weights from the population filled with the accuracies
    """
    total_accuracies = 0
    new_population = []
    
    # Get the sum of all accuracies of the population
    for individual in population:
        total_accuracies += individual[1]
        
    # For each individual, calculate its weight by dividing its accuracy by the overall sum calculated above
    for individual in population:
        weight = individual[1]/total_accuracies
        # Store the individual and its weight in the final population list
        new_population.append((individual[0], float(weight*100)))
        
    return new_population



def get_fitness_func(individual, dataframe, verbose=False):
    """
    Calculate accuracy for the individual passed as parameter.
    Both the dataframe and the y_data parameters are used for training and evaluating the model.
    """
    if verbose: print('Calculating accuracy for individual ', individual)

    from sklearn.linear_model import LogisticRegression

    X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)

    X_train = X_train.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
    X_test = X_test.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]  
    
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, y_train)

    pred_test = classifier.predict(X_test)
    return accuracy_score(y_test, pred_test)




def fill_population(individuals, dataframe, verbose=False):
    """
    Fills the population list with individuals and their weights
    """
    population = list()
    
    for individual in individuals:
        
        # Get the value of the fitness function (accuracy of the model)
        if verbose: print(f'Calculating fitness function value for individual {individual}')
        accuracy = get_fitness_func(individual, dataframe, verbose)
        
        # Check that the value is not the goal state (in this case, an accuracy of 80% is a terminal state)
        if float(accuracy) > 0.8:
            if verbose: print(f'Goal state found for individual {individual}')
            return individual
            
        individual_complete = (individual, accuracy)
        population.append(individual_complete)
        
    # The final population list is created, which contains each individual together with its weight
    # (weights will be used in the reproduction step)
    new_population = get_weights(population)
    if verbose: print(f'Generated population list (with weights): {new_population}')
    
    return new_population



def choose_parents(population, counter):
    """
    From the population, weighting the probabilities of an individual being chosen via the fitness
    function, takes randomly two individual to reproduce
    Population is a list of tuples, where the first element is the individual and the second
    one is the probability associated to it.
    To avoid generating repeated individuals, 'counter' parameter is used to pick parents in different ways, thus
    generating different individuals
    """
    # Pick random parent Number 1 and Number 2
    # (get_n_individual() function randomly picks an individual following the distribution of the weights)
    if counter == 0:        
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(1, population)
    elif counter == 1:
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(2, population)
        
    else:
        probabilities = (individual[1] for individual in population)
        individuals = [individual[0] for individual in population]
        parent_1, parent_2 = random.choices(individuals, weights=probabilities, k=2)
    
    return [parent_1, parent_2]


  
def mutate(child, prob=0.1):
    """
    Randomly mutates an individual according to the probability given by prob parameter
    """
    new_child = copy.deepcopy(child)
    for i, char in enumerate(new_child):
        if random.random() < prob:
            new_value = '1' if char == '0' else '0'
            new_child = new_child[:i] + new_value + new_child[i+1:]
    
    return new_child


  
def reproduce(individual_1, individual_2):
    """
    Takes 2 individuals, and combines their information based on a
    randomly chosen crosspoint.
    Each reproduction returns 2 new individuals
    """ 
    # Randomly generate a integer between 1 and the length of the individuals minus one, which will be the crosspoint
    crosspoint = random.randint(1, len(individual_1)-1)
    child_1 = individual_1[:crosspoint] + individual_2[crosspoint:]
    child_2 = individual_2[:crosspoint] + individual_1[crosspoint:]
    child_1, child_2 = mutate(child_1), mutate(child_2)
 
    return [child_1, child_2]


  
def generation_ahead(population, verbose=False):
    """
    Reproduces all the steps for choosing parents and making 
    childs, which means creating a new generation to iterate with
    """
    new_population = list()
    
    for _ in range(int(len(population)//2)):      
        # According to the weights calculated before, choose a set of parents to reproduce
        parents = choose_parents(population, counter=_)
        if verbose: print(f'Parents chosen: {parents}')
          
        # Reproduce the pair of individuals chose above to generate two new individuals
        childs = reproduce(parents[0], parents[1])
        if verbose: print(f'Generated children: {childs}\n')
        new_population += childs
        
    return new_population



def main_loop(ind_num, dataframe, max_iter=5, verbose=False):
    """
    Performs all the steps of the Genetic Algorithm
    1. Generate random population
    2. Fill population with the weights of each individual
    3. Check if the goal state is reached
    4. Reproduce the population, and create a new generation
    5. Repeat process until termination condition is met
    """
    # Generate individuals (returns a list of strings, where each str represents an individual)
    individuals = generate_random_individuals(ind_num, 7, 10, verbose)
    
    # Returns a list of tuples, where each tuple represents an individual and its weight
    population = fill_population(individuals, dataframe, verbose)
    
    # Check if a goal state is reached
    # When goal state is reached, fill_population() function returns a str, otherwise continue
    if isinstance(population, str):
        return population
        
    # Reproduce current generation to generate a better new one
    new_generation = generation_ahead(population, verbose)
    
    # After the new generation is generated, the loop goes on until a solution is found or until the maximum number of
    # iterations are reached
    iteration_count = 0
    while iteration_count < max_iter:
        if verbose: print(f'\n\n\nITERATION NUMBER {iteration_count+1} (Iteration max = {max_iter+1})\n\n\n')
        population = fill_population(new_generation, dataframe, verbose)
        
        # Check if a goal state is reached
        if isinstance(population, str):
            break
        
        new_generation = generation_ahead(population, verbose)   
        iteration_count += 1
        
    return population

## Create and train the model with the complete dataset

In [27]:
# y_train.isnull().sum()

In [28]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv("./processed_data.csv")

df.columns

X = df.loc[:, df.columns != 'Survived']
y = df.loc[:, df.columns == 'Survived']

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify= y)

classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

pred_test = classifier.predict(X_test)
accuracy_score(y_test, pred_test)


(891, 7) (891, 1)


/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8044692737430168

### Get Metrics

In [29]:
pred_test = classifier.predict(X_test)
accuracy_score(y_test, pred_test)

0.8044692737430168

## Create and train the model with the optimized dataset

### First, the relevant features must be selected by applying the GA

In [30]:
start = time.time()
exp_df = pd.read_csv("./processed_data.csv")
final_population = main_loop(500, exp_df, verbose=False)
print('Time elapsed for executing the recursive GA: ', time.time() - start, ' seconds')

/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

Time elapsed for executing the recursive GA:  0.11135292053222656  seconds


/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
final_population

'1111101'

### Finally, leave only the relevant features and train the model

In [32]:
optimized_dataframe = dataframe.loc[:, [True if char == '1' else False for char in '0'+final_population]]
optimized_dataframe['Survived'] = dataframe['Survived']
optimized_dataframe

/tmp/ipykernel_3455/2275766028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_dataframe['Survived'] = dataframe['Survived']


,Pclass,Sex,Age,SibSp,Parch,Embarked,Survived
0,3,1,-0.592481,1,0,2,0
1,1,0,0.638789,1,0,0,1
2,3,0,-0.284663,0,0,2,1
3,1,0,0.407926,1,0,2,1
4,3,1,0.407926,0,0,2,0
...,...,...,...,...,...,...,...
886,2,1,-0.207709,0,0,2,0
887,1,0,-0.823344,0,0,2,1
888,3,0,0.000000,1,2,2,0
889,1,1,-0.284663,0,0,0,1


In [33]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

X = optimized_dataframe.loc[:, optimized_dataframe.columns != 'Survived']
y = optimized_dataframe.loc[:, optimized_dataframe.columns == 'Survived']

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify= y)

classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

pred_test = classifier.predict(X_test)
accuracy_score(y_test, pred_test)


(891, 6) (891, 1)


/home/viZZard/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8100558659217877